<a href="https://colab.research.google.com/github/stavco9/datastreaming-final-project/blob/main/Data_Analysis_DeezyMatch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install DeezyMatch
!pip install thefuzz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 44.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7

In [2]:
import os
import tqdm
import time
from DeezyMatch import candidate_ranker, candidate_ranker_init
from thefuzz import process, fuzz

/usr/local/lib/python3.10/dist-packages/DeezyMatch/rnn_networks.py:30: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/My Drive/IDC_MSc/Year1/DataStreamingAlgorithms/DeezyMatchDataset"

Mounted at /content/drive
/content/drive/My Drive/IDC_MSc/Year1/DataStreamingAlgorithms/DeezyMatchDataset


In [4]:
words_set = []
for i in range(10):
  candidates_file = f"./candidates/dataset_candidates_{i}.txt"
  with open(candidates_file, 'r') as f:
    words_list = f.read().split('\n')
    words_set.append({"words_list": words_list})

In [5]:
some_sentance_with_error = "I wanna appel pleases or a bannana with a paer and a lemone, \
or a wartermelon if possibe. I hope i don't have an i \
don't have an errof in Londaon and in Jerusealm"

In [9]:
for index, words_list in enumerate(words_set):
    time_start_naive = time.time()
    words_set[index]["naive_matches"] = {}
    words_set[index]["naive_matches"]["origin_words"] = []
    words_set[index]["naive_matches"]["matched_words"] = []
    words_set[index]["naive_matches"]["match_scores"] = []
    for word in some_sentance_with_error.split():
      words_set[index]["naive_matches"]["origin_words"].append(word)
      word_match_results = [x for x in process.extract(word, words_list["words_list"], scorer=fuzz.ratio)]
      words_set[index]["naive_matches"]["matched_words"].append(word_match_results[0][0] if len(word_match_results) > 0 else "None")
      words_set[index]["naive_matches"]["match_scores"].append(int(word_match_results[0][1]) / 100 if len(word_match_results) > 0 else 0)
    time_end_naive = time.time()
    words_set[index]["time_naive_sec"] = (time_end_naive - time_start_naive)

In [38]:
for index in range(10):
  # Ranking on-the-fly
  # find candidates from candidate_scenario
  # for queries specified by the `query` argument
  time_start_deezy = time.time()
  ranker = \
      candidate_ranker_init(candidate_scenario=f"./combined/candidates_test_{index}",
                      query=[x.replace("-", " ") for x in some_sentance_with_error.split()],
                      ranking_metric="faiss",
                      selection_threshold=10.,
                      num_candidates=1,
                      search_size=100,
                      output_path=f"ranker_results/test_candidates_deezymatch_on_the_fly_{index}",
                      pretrained_model_path="./models/wikigaz_en_model/wikigaz_en_model.model",
                      pretrained_vocab_path="./models/wikigaz_en_model/wikigaz_en_model.vocab",
                      number_test_rows=len(some_sentance_with_error))

  ranker.rank()
  words_set[index]["deezy_match_df"] = ranker.output
  time_end_deezy = time.time()
  words_set[index]["time_deezy_sec"] = (time_end_deezy - time_start_deezy)

2024-03-23 21:22:05 a7691face663 [INFO] read input file: ./combined/candidates_test_0/train_conf.yaml
2024-03-23 21:22:05 a7691face663 [INFO] pytorch will use: cuda
2024-03-23 21:22:05 a7691face663 [INFO] use a dataframe in test_tokenize.
2024-03-23 21:22:05 a7691face663 [INFO] number of labels, True: 0 and False: 34
2024-03-23 21:22:05 a7691face663 [INFO] skipping 0 lines


2024-03-23 21:22:05 a7691face663 [INFO] save test-data-class: tmp_16cafca8-7f65-43a8-b869-cb800ecad325/query/dataframe.df


  0%|          | 0/1 [00:00<?, ?it/s]



-- Combine vectors
Reading vectors from tmp_16cafca8-7f65-43a8-b869-cb800ecad325/query/embeddings/rnn_fwd*
0000000 tmp_16cafca8-7f65-43a8-b869-cb800ecad325/query/embeddings/rnn_fwd_0


-- Combine IDs

0000000 tmp_16cafca8-7f65-43a8-b869-cb800ecad325/query/embeddings/rnn_indxs_0


-- Combine vectors
Reading vectors from tmp_16cafca8-7f65-43a8-b869-cb800ecad325/query/embeddings/rnn_bwd*
0000000 tmp_16cafca8-7f65-43a8-b869-cb800ecad325/query/embeddings/rnn_bwd_0


-- Combine IDs

0000000 tmp_16cafca8-7f65-43a8-b869-cb800ecad325/query/embeddings/rnn_indxs_0
--- 3122.386479139328 seconds ---
Is faiss_id_candis already trained? True
=========== Start the search for 0 I
ID: 1/34 -- Number of found candidates so far: 3, searched: 100
=========== Start the search for 1 wanna
ID: 2/34 -- Number of found candidates so far: 5, searched: 100
=========== Start the search for 2 appel
ID: 3/34 -- Number of found candidates so far: 30, searched: 100
=========== Start the search for 3 pleases
ID: 4/34

2024-03-23 21:22:08 a7691face663 [INFO] save test-data-class: tmp_9e4ef52f-5ca5-44df-ab79-78890c676332/query/dataframe.df


...

  0%|          | 0/1 [00:00<?, ?it/s]



-- Combine vectors
Reading vectors from tmp_64fbb2a8-9e52-49c9-8226-6fd7200e2215/query/embeddings/rnn_fwd*
0000000 tmp_64fbb2a8-9e52-49c9-8226-6fd7200e2215/query/embeddings/rnn_fwd_0


-- Combine IDs

0000000 tmp_64fbb2a8-9e52-49c9-8226-6fd7200e2215/query/embeddings/rnn_indxs_0


-- Combine vectors
Reading vectors from tmp_64fbb2a8-9e52-49c9-8226-6fd7200e2215/query/embeddings/rnn_bwd*
0000000 tmp_64fbb2a8-9e52-49c9-8226-6fd7200e2215/query/embeddings/rnn_bwd_0


-- Combine IDs

0000000 tmp_64fbb2a8-9e52-49c9-8226-6fd7200e2215/query/embeddings/rnn_indxs_0
--- 3140.246367931366 seconds ---
Is faiss_id_candis already trained? True
=========== Start the search for 0 I
ID: 1/34 -- Number of found candidates so far: 3, searched: 100
=========== Start the search for 1 wanna
ID: 2/34 -- Number of found candidates so far: 4, searched: 100
=========== Start the search for 2 appel
ID: 3/34 -- Number of found candidates so far: 29, searched: 100
=========== Start the search for 3 pleases
ID: 4/34

2024-03-23 21:22:26 a7691face663 [INFO] save test-data-class: tmp_cdbee345-6332-410e-8b36-aa377ada061c/query/dataframe.df


  0%|          | 0/1 [00:00<?, ?it/s]



-- Combine vectors
Reading vectors from tmp_cdbee345-6332-410e-8b36-aa377ada061c/query/embeddings/rnn_fwd*
0000000 tmp_cdbee345-6332-410e-8b36-aa377ada061c/query/embeddings/rnn_fwd_0


-- Combine IDs

0000000 tmp_cdbee345-6332-410e-8b36-aa377ada061c/query/embeddings/rnn_indxs_0


-- Combine vectors
Reading vectors from tmp_cdbee345-6332-410e-8b36-aa377ada061c/query/embeddings/rnn_bwd*
0000000 tmp_cdbee345-6332-410e-8b36-aa377ada061c/query/embeddings/rnn_bwd_0


-- Combine IDs

0000000 tmp_cdbee345-6332-410e-8b36-aa377ada061c/query/embeddings/rnn_indxs_0
--- 3142.8159770965576 seconds ---
Is faiss_id_candis already trained? True
=========== Start the search for 0 I
ID: 1/34 -- Number of found candidates so far: 2, searched: 100
=========== Start the search for 1 wanna
ID: 2/34 -- Number of found candidates so far: 3, searched: 100
=========== Start the search for 2 appel
ID: 3/34 -- Number of found candidates so far: 33, searched: 100
=========== Start the search for 3 pleases
ID: 4/3

In [35]:
for index in range(10):
    deezy_match_df = words_set[index]["deezy_match_df"]
    words_set[index]["deezy_matches"] = {}
    words_set[index]["deezy_matches"]["origin_words"] = []
    words_set[index]["deezy_matches"]["matched_words"] = []
    words_set[index]["deezy_matches"]["match_distance"] = []
    for word in some_sentance_with_error.split():
      word_df_row = deezy_match_df.loc[deezy_match_df['query'] == word].iloc[0]
      words_set[index]["deezy_matches"]["origin_words"].append(word)
      words_set[index]["deezy_matches"]["matched_words"].append(list(word_df_row['faiss_distance'].keys())[0] if bool(word_df_row['faiss_distance']) else None)
      words_set[index]["deezy_matches"]["match_distance"].append(list(word_df_row['faiss_distance'].values())[0] if bool(word_df_row['faiss_distance']) else -1)

In [36]:
for index, words_list in enumerate(words_set):
   print(f"Total time of dataset {index + 1} is {words_list['time_naive_sec']}s")
   print(f"Origin words for dataset {index + 1} are {words_list['naive_matches']['origin_words']}")
   print(f"Matches words for dataset {index + 1} are {words_list['naive_matches']['matched_words']}")
   print(f"Matches scores for dataset {index + 1} are {words_list['naive_matches']['match_scores']}")

Total time of dataset 1 is 6.014449119567871s
Origin words for dataset 1 are ['I', 'wanna', 'appel', 'pleases', 'or', 'a', 'bannana', 'with', 'a', 'paer', 'and', 'a', 'lemone,', 'or', 'a', 'wartermelon', 'if', 'possibe.', 'I', 'hope', 'i', "don't", 'have', 'an', 'i', "don't", 'have', 'an', 'errof', 'in', 'Londaon', 'and', 'in', 'Jerusealm']
Matches words for dataset 1 are ['I', 'Wanna', 'appeal', 'Please', 'or', 'a', 'bananas', 'with', 'a', 'paper', 'and', 'a', 'Lemon', 'or', 'a', 'Waterloo', 'if', 'possible?!', 'I', 'Hope.', 'I', "don't", 'have', 'an', 'I', "don't", 'have', 'an', 'error', 'in', 'London', 'and', 'in', 'Jerusalem']
Matches scores for dataset 1 are [1.0, 1.0, 0.91, 0.92, 1.0, 1.0, 0.86, 1.0, 1.0, 0.89, 1.0, 1.0, 0.91, 1.0, 1.0, 0.74, 1.0, 0.93, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.8, 1.0, 0.92, 1.0, 1.0, 0.89]
Total time of dataset 2 is 5.98909068107605s
Origin words for dataset 2 are ['I', 'wanna', 'appel', 'pleases', 'or', 'a', 'bannana', 'with', 'a', 'p

In [39]:
for index in range(10):
   print(f"Total time of dataset {index + 1} is {words_set[index]['time_deezy_sec']}s")
   print(f"Origin words for dataset {index + 1} are {words_set[index]['deezy_matches']['origin_words']}")
   print(f"Matches words for dataset {index + 1} are {words_set[index]['deezy_matches']['matched_words']}")
   print(f"Matches scores for dataset {index + 1} are {words_set[index]['deezy_matches']['match_distance']}")

Total time of dataset 1 is 2.1905744075775146s
Origin words for dataset 1 are ['I', 'wanna', 'appel', 'pleases', 'or', 'a', 'bannana', 'with', 'a', 'paer', 'and', 'a', 'lemone,', 'or', 'a', 'wartermelon', 'if', 'possibe.', 'I', 'hope', 'i', "don't", 'have', 'an', 'i', "don't", 'have', 'an', 'errof', 'in', 'Londaon', 'and', 'in', 'Jerusealm']
Matches words for dataset 1 are ['I', 'Wanna', 'papel', 'Settles', 'or', 'a', 'bandanna', 'with', 'a', 'Parker', 'and', 'a', 'Lemon', 'or', 'a', 'trawled', 'if', 'possible.', 'I', 'Hope', 'I', "Don't", 'have', 'an', 'I', "Don't", 'have', 'an', 'error', 'in', 'London', 'and', 'in', 'Jerusalem']
Matches scores for dataset 1 are [0.0, 0.0, 4.7146, 5.0393, 0.0, 0.0, 3.2076, 0.0, 0.0, 3.4475, 0.0, 0.0, 6.6457, 0.0, 0.0, 7.4863, 0.0, 4.5969, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.2706, 0.0, 3.8871, 0.0, 0.0, 5.5064]
Total time of dataset 2 is 2.1793768405914307s
Origin words for dataset 2 are ['I', 'wanna', 'appel', 'pleases', 'or', 'a', 'ba